In [ ]:
import com.google.gson.Gson
import hu.raven.puppet.job.experiments.CostStatistics
import java.io.File

val inputFolder = File("D:\\Git\\GitHub\\SourceCodes\\Kotlin\\BacterialMemeticConsole\\output")
val costStatistics = inputFolder.listFiles()
    .map { file ->
        println(file)
        Gson().fromJson<CostStatistics>(file.readLines().joinToString("\n"), CostStatistics::class.java)
    }

In [ ]:
val initialGapStatistics = costStatistics.map { it.initialGap }
val builtGapStatistics = costStatistics.map { it.builtGap }
val optimizedGapStatistics = costStatistics.map { it.optimizedGap }

val data = mapOf(
    "phase" to List(6) { "initial"} + List(6) { "local search"} +List(6) { "final"} ,
    "Q1" to initialGapStatistics.map { it.q1 * 100 } + builtGapStatistics.map { it.q1 * 100} + optimizedGapStatistics.map { it.q1 * 100},
    "median" to initialGapStatistics.map { it.median * 100} + builtGapStatistics.map { it.median* 100 } + optimizedGapStatistics.map { it.median * 100},
    "Q3" to initialGapStatistics.map { it.q2 * 100} + builtGapStatistics.map { it.q2* 100 } + optimizedGapStatistics.map { it.q2* 100 },
)

In [ ]:
%use kandy

In [ ]:
data.plot {
    layout {
        title = "Gap of new solution in the three phases"
        guideLegend()
    }

    x(
        listOf("0.01", "0.02", "0.04", "0.08", "0.16", "0.32") +
                listOf("0.01", "0.02", "0.04", "0.08", "0.16", "0.32") +
                listOf("0.01", "0.02", "0.04", "0.08", "0.16", "0.32")
    ) { axis.name = "noise deviation" }
    y {
        axis.name = "gap%"
        scale = continuous(0.0..70.0)
    }

    groupBy("phase") {
        ribbon {
            yMin("Q1")
            yMax("Q3")
            fillColor = Color.BLACK
            alpha = 0.25
            borderLine {
                borderLine.color = Color.GREEN
                width = 0.0
            }
        }

        line {
            y("median")
            color("phase") {
                scale = categorical("final" to Color.BLACK, "local search" to Color.BLACK, "initial" to Color.BLACK)
            }
            width = 1.5
            type("phase") {
                scale = categorical(
                    "final" to LineType.DASHED, 
                    "local search" to LineType.DOTTED,
                    "initial" to LineType.TWODASH
                )
                color= Color.BLACK 
            }
        }
    }
}

In [ ]:
import hu.akos.hollo.szabo.collections.asImmutable
import hu.akos.hollo.szabo.math.asDoubleVector
import hu.akos.hollo.szabo.math.matrix.DoubleMatrix

var matrix = File("D:\\Git\\GitHub\\SourceCodes\\Kotlin\\BacterialMemeticConsole\\array\\array_data.txt").readLines()
    .map {
        it
            .split(" ")
            .map { it.toDouble() }
            .toDoubleArray()
            .asDoubleVector()
    }
    .toTypedArray()
    .asImmutable()
    .let { DoubleMatrix(it) }

In [ ]:
matrix = matrix.indices.last()
    .map {
        matrix.getRow(it)
    }
    .toTypedArray()
    .asImmutable()
    .let { DoubleMatrix(it) }

In [ ]:
matrix.dimensions

In [ ]:
import hu.raven.puppet.job.experiments.statistics

val statistics = matrix.columnVectors
    .map { it.asDoubleArray().statistics() }

In [ ]:
statistics.size

In [ ]:
val newData = mapOf(
    "phase" to List(6) { listOf("final","local search","initial")}.flatten() ,
    "Q1" to statistics.map { it.q1  } ,
    "median" to statistics.map { it.median } ,
    "Q3" to statistics.map { it.q2 },
)

In [ ]:
newData.plot {
    layout {
        title = "Gap of original solution in the three phases"
    }

    x(
        listOf("0.01", "0.02", "0.04", "0.08", "0.16", "0.32") +
                listOf("0.01", "0.02", "0.04", "0.08", "0.16", "0.32") +
                listOf("0.01", "0.02", "0.04", "0.08", "0.16", "0.32")
    ) { axis.name = "noise deviation" }
    y {
        axis.name = "gap%"
        axis.breaks(List(9/4*5 + 2){it*20})
    }

    groupBy("phase") {
        ribbon {
            yMin("Q1")
            yMax("Q3")
            fillColor = Color.BLACK
            alpha = 0.25
            borderLine {
                borderLine.color = Color.GREEN
                width = 0.0
            }
        }

        line {
            y("median")
            color("phase") {
                scale = categorical("final" to Color.BLACK, "local search" to Color.BLACK, "initial" to Color.BLACK)
            }
            width = 1.5
            type("phase") {
                scale = categorical(
                    "final" to LineType.DASHED,
                    "local search" to LineType.DOTTED,
                    "initial" to LineType.TWODASH
                )
                color= Color.BLACK
            }
        }
    }
}